# Calculate biomass from deadwood

# Imports and Set-up

In [1]:
# Standard Imports
import sys
import pandas as pd
import numpy as np

# Google Cloud Imports
import pandas_gbq

In [68]:
# Util imports
sys.path.append("../../")  # include parent directory
from src.settings import (
    GCP_PROJ_ID,
    CARBON_POOLS_OUTDIR,
    CARBON_STOCK_OUTDIR,
    SPECIES_LOOKUP_CSV,
    PC_PLOT_LOOKUP_CSV,
    TMP_OUT_DIR,
)

from src.biomass_equations import (
    vmd0002_eq1,
    vmd0002_eq2,
    vmd0002_eq3,
    vmd0002_eq4,
    vmd0002_eq7,
    vmd0002_eq8a,
    vmd0002_eq8b,
    vmd0002_eq9,
    get_solid_diamter,
    calculate_tree_height,
    allometric_tropical_tree,
    allometric_peatland_tree,
)

In [3]:
# Variables
PLOT_INFO_CSV = CARBON_POOLS_OUTDIR / "plot_info.csv"
STUMPS_CSV = CARBON_POOLS_OUTDIR / "stumps.csv"
LDW_CSV = CARBON_POOLS_OUTDIR / "lying_deadwood_wo_hollow.csv"
LDW_HOLLOW_CSV = CARBON_POOLS_OUTDIR / "lying_deadwood_hollow.csv"
DEAD_TREES_CSV = CARBON_POOLS_OUTDIR / "dead_trees.csv"

# Temporary Output Files
tmp_dead_trees_c1 = TMP_OUT_DIR / "c1_dead_trees.csv"
tmp_dead_trees_c1_wd = TMP_OUT_DIR / "c1_dead_trees_wd.csv"

# BigQuery Variables
SRC_DATASET_ID = "biomass_inventory"
DATASET_ID = "carbon_stock"
IF_EXISTS = "replace"

## Load data

### Plot Data

In [4]:
if PLOT_INFO_CSV.exists():
    plot_info = pd.read_csv(PLOT_INFO_CSV)
else:
    query = f"""
    SELECT
        * 
    FROM {GCP_PROJ_ID}.{SRC_DATASET_ID}.plot_info"""

    # Read the BigQuery table into a dataframe
    plot_info = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    plot_info.to_csv(PLOT_INFO_CSV, index=False)

In [5]:
plot_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   unique_id                  671 non-null    object 
 1   data_recorder              671 non-null    object 
 2   team_no                    671 non-null    int64  
 3   plot_code_nmbr             671 non-null    int64  
 4   plot_type                  671 non-null    object 
 5   sub_plot                   671 non-null    object 
 6   yes_no                     671 non-null    object 
 7   sub_plot_shift             633 non-null    object 
 8   GPS_waypt                  633 non-null    float64
 9   GPS_id                     633 non-null    float64
 10  GPS                        576 non-null    object 
 11  GPS_latitude               576 non-null    float64
 12  GPS_longitude              576 non-null    float64
 13  GPS_altitude               576 non-null    float64

In [6]:
# get the slope adjusted area per nest per subplot and creaste dict for substitution
plot_info_subset = plot_info[
    [
        "unique_id",
        "corrected_plot_area_n2_m2",
        "corrected_plot_area_n3_m2",
        "corrected_plot_area_n4_m2",
    ]
].copy()
plot_info_subset.dropna(inplace=True)
plot_info_subset.drop_duplicates(subset=["unique_id"], inplace=True)
plot_info_subset_dict = plot_info_subset.to_dict(orient="records")

### Stumps

In [7]:
if STUMPS_CSV.exists():
    stumps = pd.read_csv(STUMPS_CSV)
else:
    query = f"""
    SELECT
        * 
    FROM {GCP_PROJ_ID}.{DATASET_ID}.stumps"""

    # Read the BigQuery table into a dataframe
    stumps = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    stumps.to_csv(STUMPS_CSV, index=False)

In [8]:
stumps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754 entries, 0 to 1753
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   unique_id      1754 non-null   object 
 1   nest           1754 non-null   int64  
 2   Diam1          1754 non-null   float64
 3   Diam2          1754 non-null   float64
 4   slope          1754 non-null   float64
 5   height         1754 non-null   float64
 6   cut_cl         1754 non-null   object 
 7   hollow_go      1754 non-null   object 
 8   hollow_d1      171 non-null    float64
 9   hollow_d2      171 non-null    float64
 10  stump_density  1754 non-null   float64
dtypes: float64(7), int64(1), object(3)
memory usage: 150.9+ KB


### Lying deadwood

In [9]:
if LDW_CSV.exists():
    ldw = pd.read_csv(LDW_CSV)
else:
    query = f"""
    SELECT
        * 
    FROM {GCP_PROJ_ID}.{SRC_DATASET_ID}.lying_deadwood_wo_hollow"""

    # Read the BigQuery table into a dataframe
    ldw = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    ldw.to_csv(LDW_CSV, index=False)

In [10]:
ldw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   unique_id   1826 non-null   object 
 1   repetition  1826 non-null   int64  
 2   type        1826 non-null   object 
 3   class       1826 non-null   object 
 4   diameter    1826 non-null   float64
 5   density     1826 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 85.7+ KB


In [11]:
if LDW_HOLLOW_CSV.exists():
    ldw_hollow = pd.read_csv(LDW_HOLLOW_CSV)
else:
    query = f"""
    SELECT
        * 
    FROM {GCP_PROJ_ID}.{SRC_DATASET_ID}.lying_deadwood_hollow"""

    # Read the BigQuery table into a dataframe
    ldw_hollow = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    ldw_hollow.to_csv(LDW_HOLLOW_CSV, index=False)

In [12]:
ldw_hollow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   unique_id   15 non-null     object 
 1   repetition  15 non-null     int64  
 2   type        15 non-null     object 
 3   class       15 non-null     object 
 4   hollow_d1   15 non-null     float64
 5   hollow_d2   15 non-null     float64
 6   diameter    15 non-null     float64
 7   density     15 non-null     float64
dtypes: float64(4), int64(1), object(3)
memory usage: 1.1+ KB


### Standing Deadwood

In [13]:
if DEAD_TREES_CSV.exists():
    dead_trees = pd.read_csv(DEAD_TREES_CSV)
else:
    query = f"""
    SELECT
        * 
    FROM {GCP_PROJ_ID}.{SRC_DATASET_ID}.dead_trees"""

    # Read the BigQuery table into a dataframe
    dead_trees = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    dead_trees.to_csv(DEAD_TREES_CSV, index=False)

In [14]:
dead_trees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   unique_id     256 non-null    object 
 1   nest          256 non-null    int64  
 2   species_name  203 non-null    float64
 3   DBH_cl1       2 non-null      float64
 4   class         256 non-null    int64  
 5   subclass      254 non-null    object 
 6   family_name   12 non-null     float64
 7   dbh_tall      254 non-null    float64
 8   db_tall       254 non-null    float64
 9   tall_density  254 non-null    float64
 10  slope_t_tall  254 non-null    float64
 11  slope_b_tall  254 non-null    float64
 12  dist_t_tall   254 non-null    float64
dtypes: float64(9), int64(2), object(2)
memory usage: 26.1+ KB


### Tree species

In [15]:
species = pd.read_csv(SPECIES_LOOKUP_CSV)

In [16]:
species.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   code_family      74 non-null     float64
 1   family           74 non-null     object 
 2   scientific_name  375 non-null    object 
 3   local_name       375 non-null    object 
 4   code_species     375 non-null    int64  
 5   corrected_genus  375 non-null    object 
 6   wood_density     375 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 20.6+ KB


In [17]:
species.head(2)

,code_family,family,scientific_name,local_name,code_species,corrected_genus,wood_density
0,999.0,Unknown,Litchi chinensis,Alupag - amo,193,Litchi,0.608902
1,1.0,Alangiaceae,Alangium javanicum,Putian,15,Alangium,0.608902


In [18]:
species_dict = (
    species[["scientific_name", "code_species"]]
    .set_index("code_species")
    .to_dict()["scientific_name"]
)

In [19]:
# create lookup table for family name and code
species_family = species[["code_family", "family"]].drop_duplicates()

In [20]:
family_dict = species_family.set_index("code_family").to_dict()["family"]

### Plot lookup

In [21]:
plot_strata = pd.read_csv(PC_PLOT_LOOKUP_CSV)

In [22]:
plot_strata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3508 entries, 0 to 3507
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   unique_id  3508 non-null   object
 1   Strata     3508 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 54.9+ KB


# Calculate stump biomass

In [23]:
stumps.head(2)

,unique_id,nest,Diam1,Diam2,slope,height,cut_cl,hollow_go,hollow_d1,hollow_d2,stump_density
0,308C1,2,30.0,29.0,43.0,18.0,saw_axe,no,NaN,NaN,1.0
1,249B1,2,15.0,10.0,51.0,80.0,saw_axe,no,NaN,NaN,3.0


In [24]:
# get wood density equivalent for each density class

density_val = {1: 0.54, 2: 0.35, 3: 0.21}
stumps["stump_density_val"] = stumps["stump_density"].replace(density_val).fillna(0.21)

In [25]:
# convert height from cm to m
stumps["height_m"] = stumps["height"] / 100

In [26]:
# Get biomass for each stump
stumps = vmd0002_eq2(stumps, "Diam1", "Diam2", "height_m", "stump_density_val")

In [27]:
# Get biomass of each stump that is hollow
stumps_hollow = vmd0002_eq2(
    stumps, "hollow_d1", "hollow_d2", "height_m", "stump_density_val"
)

In [28]:
# Get biomass to subtract due to hollow stumps
stumps["tonnes_dry_matter_hollow"] = stumps_hollow["tonnes_dry_matter"]

In [29]:
# Subtract biomass of hollow stumps from total biomass
stumps["tonnes_dry_matter"] = np.where(
    (~stumps["tonnes_dry_matter_hollow"].isna())
    & (stumps["tonnes_dry_matter_hollow"] > 0),
    stumps["tonnes_dry_matter"] - stumps["tonnes_dry_matter_hollow"],
    stumps["tonnes_dry_matter"],
)

In [30]:
stumps.describe()

,nest,Diam1,Diam2,slope,height,hollow_d1,hollow_d2,stump_density,stump_density_val,height_m,tonnes_dry_matter,tonnes_dry_matter_hollow
count,1754.000000,1754.000000,1754.000000,1754.000000,1754.000000,171.000000,171.000000,1754.000000,1754.000000,1754.000000,1754.000000,171.000000
mean,3.111745,40.575610,36.617423,32.697834,83.215314,36.059532,33.773450,2.128848,0.344818,0.832153,0.106916,0.101488
std,0.636456,28.664704,24.662826,18.110894,40.670748,23.908597,21.810063,0.791743,0.129731,0.406707,0.114977,0.090179
min,2.000000,10.000000,10.000000,1.000000,1.500000,5.000000,5.000000,1.000000,0.210000,0.015000,-0.013249,0.003024
25%,3.000000,20.000000,18.700000,16.000000,52.000000,18.550000,18.250000,1.000000,0.210000,0.520000,0.034926,0.033507
50%,3.000000,32.000000,30.000000,33.000000,80.000000,30.000000,30.000000,2.000000,0.350000,0.800000,0.069472,0.071736
75%,4.000000,53.725000,48.875000,47.000000,110.000000,48.000000,43.000000,3.000000,0.540000,1.100000,0.136500,0.150080
max,4.000000,195.000000,198.000000,80.000000,199.100000,160.000000,150.000000,3.000000,0.540000,1.991000,0.994194,0.487305


In [31]:
# Remove biomass_hollow column to avoid confusion
stumps.drop(columns=["tonnes_dry_matter_hollow"], inplace=True)

In [32]:
stumps.head(2)

,unique_id,nest,Diam1,Diam2,slope,height,cut_cl,hollow_go,hollow_d1,hollow_d2,stump_density,stump_density_val,height_m,tonnes_dry_matter
0,308C1,2,30.0,29.0,43.0,18.0,saw_axe,no,NaN,NaN,1.0,0.54,0.18,0.028674
1,249B1,2,15.0,10.0,51.0,80.0,saw_axe,no,NaN,NaN,3.0,0.21,0.80,0.021000


## Get total stump biomass per hectare per subplot

In [33]:
# get sum opf dry matter per subplot
stumps_agg = vmd0002_eq3(stumps, ["unique_id", "nest"], "tonnes_dry_matter")

In [34]:
# add the correct area using the unique_id and nest number
stumps_agg["corrected_area_m2"] = stumps_agg.apply(
    lambda x: next(
        (
            item["corrected_plot_area_n" + str(x["nest"]) + "_m2"]
            for item in plot_info_subset_dict
            if item["unique_id"] == x["unique_id"]
        ),
        None,
    ),
    axis=1,
)

In [35]:
# convert square meters to hectares
stumps_agg["corrected_area_ha"] = stumps_agg["corrected_area_m2"] / 10_000

In [36]:
stumps_agg = vmd0002_eq4(stumps_agg, "tonnes_dry_matter", "corrected_area_ha")

In [37]:
stumps_agg.head(2)

,unique_id,nest,tonnes_dry_matter,corrected_area_m2,corrected_area_ha,tonnes_dry_matter_ha
0,100C1,3,0.11403,707.141090,0.070714,1.612549
1,100D1,3,0.11307,779.240642,0.077924,1.451028


In [38]:
stumps_agg.rename(
    columns={"tonnes_dry_matter_ha": "stumps_tonnes_dry_matter_ha"}, inplace=True
)

In [39]:
stumps_agg = (
    stumps_agg[["unique_id", "stumps_tonnes_dry_matter_ha"]]
    .groupby("unique_id")
    .mean()
    .reset_index()
)

In [40]:
stumps_agg.head(2)

,unique_id,stumps_tonnes_dry_matter_ha
0,100C1,1.612549
1,100D1,1.216131


# Calculate Lying deadwood biomass

In [41]:
ldw.head(2)

,unique_id,repetition,type,class,diameter,density
0,308D1,1,tr1,FC,16.5,3.0
1,308D1,2,tr1,FC,18.3,3.0


## No hollow

### Outlier removal

In [42]:
ldw.describe()

,repetition,diameter,density
count,1826.000000,1826.000000,1826.000000
mean,2.529573,27.733866,1.937021
std,1.943715,109.834565,0.744572
min,1.000000,10.000000,1.000000
25%,1.000000,14.425000,1.000000
50%,2.000000,20.000000,2.000000
75%,3.000000,30.000000,2.000000
max,15.000000,4592.000000,3.000000


In [43]:
# Filter the ldw DataFrame to keep rows where diameter is less than or equal to the 98th percentile
ldw = ldw[ldw["diameter"] <= 150]

In [44]:
ldw = vmd0002_eq7(ldw, "diameter", 80)

In [45]:
ldw = vmd0002_eq8a(ldw, "density")

In [46]:
ldw.head(2)

,unique_id,repetition,type,class,diameter,density,deadwood_volume,tonnes_dry_matter_ha
0,308D1,1,tr1,FC,16.5,3.0,0.425391,0.089332
1,308D1,2,tr1,FC,18.3,3.0,0.523266,0.109886


## Hollow Lying Deadwood

In [47]:
ldw_hollow = get_solid_diamter(ldw_hollow, "hollow_d1", "hollow_d2", "diameter")

In [48]:
ldw_hollow = vmd0002_eq7(ldw_hollow, "solid_diameter", 80)

In [49]:
ldw_hollow = vmd0002_eq8a(ldw_hollow, "density")

In [50]:
ldw_hollow.head(2)

,unique_id,repetition,type,class,hollow_d1,hollow_d2,diameter,density,solid_diameter,deadwood_volume,tonnes_dry_matter_ha
0,249D1,1,tr2,MDF,12.0,10.0,16.7,3.0,12.565429,0.246703,0.051808
1,290A1,2,tr2,MCB,68.0,28.0,62.0,1.0,39.242834,2.406250,1.299375


## Get total lying deadwood biomass per hectare per subplot

In [51]:
ldw_subset = ldw[["unique_id", "tonnes_dry_matter_ha"]].copy()
ldw_hollow_subset = ldw_hollow[["unique_id", "tonnes_dry_matter_ha"]].copy()

In [52]:
ldw_all = pd.concat([ldw_subset, ldw_hollow_subset])

In [53]:
ldw_all.head(2)

,unique_id,tonnes_dry_matter_ha
0,308D1,0.089332
1,308D1,0.109886


In [54]:
ldw_agg = vmd0002_eq8b(ldw_all, agg_col=["unique_id"])

In [55]:
ldw_agg.rename(
    columns={"tonnes_dry_matter_ha": "ldw_tonnes_dry_matter_ha"}, inplace=True
)

In [56]:
ldw_agg.head(2)

,unique_id,ldw_tonnes_dry_matter_ha
0,100A1,0.207976
1,100B1,1.091449


In [57]:
ldw_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476 entries, 0 to 475
Data columns (total 2 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   unique_id                 476 non-null    object 
 1   ldw_tonnes_dry_matter_ha  476 non-null    float64
dtypes: float64(1), object(1)
memory usage: 7.6+ KB


# Calculate Standing Deadwood Biomass

## Calculate biomass for Class 1 standing deadwood trees
Class 1 standing dead trees that is fresh and can be treated as living trees in terms of biomass. The method applied mimics the process from living trees

In [58]:
dead_trees.head(2)

,unique_id,nest,species_name,DBH_cl1,class,subclass,family_name,dbh_tall,db_tall,tall_density,slope_t_tall,slope_b_tall,dist_t_tall
0,290C1,3,145.0,38.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,290C1,4,177.0,58.8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
dead_trees.rename(
    columns={"family_name": "code_family", "species_name": "code_species"}, inplace=True
)

In [60]:
c1_dead_trees = dead_trees.loc[dead_trees["class"] == 1].copy()

In [61]:
c1_dead_trees["family_name"] = c1_dead_trees["code_family"].replace(family_dict)

In [62]:
c1_dead_trees["scientific_name"] = c1_dead_trees["code_species"].replace(species_dict)

In [63]:
c1_dead_trees.to_csv(tmp_dead_trees_c1)

In [64]:
c1_dead_trees

,unique_id,nest,code_species,DBH_cl1,class,subclass,code_family,dbh_tall,db_tall,tall_density,slope_t_tall,slope_b_tall,dist_t_tall,family_name,scientific_name
0,290C1,3,145.0,38.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Flueggea flexuosa
1,290C1,4,177.0,58.8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hopea plagata


### Get genus and wood density using BIOMASS R Library

In [69]:
!Rscript $SRC_DIR"/get_wood_density.R" $tmp_dead_trees_c1 $tmp_dead_trees_c1_wd

Using user data cache /Users/renflores/Library/Application Support/R/BIOMASS
  To clear or remove cache see function clearCache().
Using useCache=TRUE is recommended to reduce online search time for the next query
  |======================================================================| 100%
Source iplant_tnrs:2
Corrections FALSE:2
The reference dataset contains 16467 wood density values
Your taxonomic table contains 2 taxa
Error in getWoodDensity(genus = data$corrected_genus, species = data$scientific_name,  : 
  There is no exact match among the family, genus and species, try with 'addWoodDensity'
         or inform the 'family' or increase the 'region'
Execution halted


In [70]:
c1_dead_trees = pd.read_csv(tmp_dead_trees_c1_wd)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/renflores/Documents/OneBase/notebooks/02_carbon_stock/../../data/tmp/c1_dead_trees_wd.csv'

In [71]:
c1_dead_trees

,unique_id,nest,code_species,DBH_cl1,class,subclass,code_family,dbh_tall,db_tall,tall_density,slope_t_tall,slope_b_tall,dist_t_tall,family_name,scientific_name
0,290C1,3,145.0,38.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Flueggea flexuosa
1,290C1,4,177.0,58.8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hopea plagata


In [72]:
c1_dead_trees["wood_density"] = np.nan

In [74]:
c1_dead_trees.loc[c1_dead_trees.scientific_name == "Hopea plagata", "wood_density"] = (
    0.64
)

In [75]:
c1_dead_trees.loc[
    c1_dead_trees.scientific_name == "Flueggea flexuosa", "wood_density"
] = 0.64

In [76]:
c1_dead_trees = calculate_tree_height(c1_dead_trees, "DBH_cl1")

In [77]:
c1_dead_trees = c1_dead_trees.merge(
    plot_strata[["unique_id", "Strata"]], on="unique_id", how="left"
)

In [78]:
c1_dead_trees_tropical = c1_dead_trees.loc[
    c1_dead_trees["Strata"].isin([1, 2, 3])
].copy()

In [79]:
c1_dead_trees_tropical = allometric_tropical_tree(
    c1_dead_trees_tropical, "wood_density", "DBH_cl1", "height"
)

In [80]:
c1_dead_trees_peatland = c1_dead_trees.loc[
    c1_dead_trees["Strata"].isin([4, 5, 6])
].copy()

In [81]:
c1_dead_trees_peatland = allometric_peatland_tree(c1_dead_trees_peatland, "DBH_cl1")

In [82]:
c1_dead_trees = pd.concat([c1_dead_trees_tropical, c1_dead_trees_peatland])

In [83]:
c1_dead_trees.drop(columns=["X"], inplace=True)

KeyError: "['X'] not found in axis"

In [84]:
c1_dead_trees

,unique_id,nest,code_species,DBH_cl1,class,subclass,code_family,dbh_tall,db_tall,tall_density,slope_t_tall,slope_b_tall,dist_t_tall,family_name,scientific_name,wood_density,height,Strata,aboveground_biomass
0,290C1,3,145.0,38.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Flueggea flexuosa,0.64,25.481783,2,1244.699729
1,290C1,4,177.0,58.8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hopea plagata,0.64,30.000000,2,3422.485398


In [85]:
c1_dead_trees["tonnes_dry_matter"] = c1_dead_trees["aboveground_biomass"] / 1_000

In [86]:
# vmd0001_eq5(c1_dead_trees)

## Calculate biomass for Class 2 standing deadwood short trees
no short trees (that are not stumps) in the dataset. add in steps when data is available

## Calculate biomass for Class 2  standing deadwood tall trees

Class 2 are standing dead trees with assigned density class

In [87]:
c2_dead_trees_t = dead_trees.loc[
    (dead_trees["class"] == 2) & (dead_trees["subclass"] == "tall")
].copy()

In [88]:
# convert slope to radians
c2_dead_trees_t["slope_t_tall_radians"] = np.atan(c2_dead_trees_t["slope_t_tall"]) / 100
c2_dead_trees_t["slope_b_tall_radians"] = np.atan(c2_dead_trees_t["slope_b_tall"]) / 100

In [89]:
# estimate tree height
c2_dead_trees_t = calculate_tree_height(
    c2_dead_trees_t,
    trig_leveling=True,
    dist_col="dist_t_tall",
    slope_b_col="slope_b_tall_radians",
    slope_t_col="slope_t_tall_radians",
)

In [90]:
# set wood density equivalent for each density class
density_val = {1: 0.54, 2: 0.35, 3: 0.21}
c2_dead_trees_t["density_val"] = (
    c2_dead_trees_t["tall_density"].replace(density_val).fillna(0.21)
)

In [91]:
c2_dead_trees_t = vmd0002_eq1(c2_dead_trees_t, "db_tall", "height", "density_val")

In [92]:
c2_dead_trees_t

,unique_id,nest,code_species,DBH_cl1,class,subclass,code_family,dbh_tall,db_tall,tall_density,slope_t_tall,slope_b_tall,dist_t_tall,slope_t_tall_radians,slope_b_tall_radians,height,density_val,tonnes_dry_matter
2,368A1,2,999.0,NaN,2,tall,NaN,21.9,24.1,1.0,61.0,56.0,10.0,0.015544,0.015529,0.000146,0.54,0.000001
3,281A1,2,NaN,NaN,2,tall,NaN,32.6,40.8,1.0,115.0,20.0,9.3,0.015621,0.015208,0.003837,0.54,0.000090
4,214C1,2,52.0,NaN,2,tall,NaN,21.2,23.5,1.0,82.0,-30.0,6.8,0.015586,-0.015375,0.210668,0.54,0.001645
5,227D1,2,NaN,NaN,2,tall,NaN,25.5,30.2,2.0,62.0,13.0,5.0,0.015547,0.014940,0.003032,0.35,0.000025
6,336C1,2,NaN,NaN,2,tall,NaN,10.5,14.6,2.0,41.0,14.0,5.0,0.015464,0.014995,0.002346,0.35,0.000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,174C1,4,999.0,NaN,2,tall,NaN,52.0,64.0,3.0,102.0,15.0,10.0,0.015610,0.015042,0.005676,0.21,0.000128
252,111C1,4,999.0,NaN,2,tall,NaN,62.3,59.7,2.0,47.0,16.0,10.0,0.015495,0.015084,0.004115,0.35,0.000134
253,47D1,4,999.0,NaN,2,tall,NaN,50.2,60.0,2.0,80.0,18.0,10.0,0.015583,0.015153,0.004300,0.35,0.000142
254,5B1,4,287.0,NaN,2,tall,NaN,85.2,102.8,2.0,20.0,48.0,10.0,0.015208,0.015500,0.002913,0.35,0.000282


### combine class 1 and class 2

In [93]:
c2_subset = c2_dead_trees_t[["unique_id", "nest", "tonnes_dry_matter"]].copy()
c1_subset = c1_dead_trees[["unique_id", "nest", "tonnes_dry_matter"]].copy()

In [94]:
sdw_all = pd.concat([c1_subset, c2_subset])

In [95]:
sdw_all.head(2)

,unique_id,nest,tonnes_dry_matter
0,290C1,3,1.244700
1,290C1,4,3.422485


## get total standing deadwood biomass 

In [96]:
sdw_all_agg = vmd0002_eq3(sdw_all, ["unique_id", "nest"], "tonnes_dry_matter")

In [97]:
# add the correct area using the unique_id and nest number
sdw_all_agg["corrected_area_m2"] = sdw_all_agg.apply(
    lambda x: next(
        (
            item["corrected_plot_area_n" + str(x["nest"]) + "_m2"]
            for item in plot_info_subset_dict
            if item["unique_id"] == x["unique_id"]
        ),
        None,
    ),
    axis=1,
)

In [98]:
# convert square meters to hectares
sdw_all_agg["corrected_area_ha"] = sdw_all_agg["corrected_area_m2"] / 10_000

In [99]:
sdw_all_agg = vmd0002_eq4(sdw_all_agg, "tonnes_dry_matter", "corrected_area_ha")

In [100]:
sdw_all_agg

,unique_id,nest,tonnes_dry_matter,corrected_area_m2,corrected_area_ha,tonnes_dry_matter_ha
0,110C1,2,0.000011,102.298111,0.010230,0.001046
1,110C1,3,0.000073,920.682997,0.092068,0.000796
2,110D1,2,0.000027,94.444129,0.009444,0.002858
3,111C1,3,0.000289,708.625493,0.070863,0.004084
4,111C1,4,0.000134,1259.778654,0.125978,0.001067
...,...,...,...,...,...,...
208,97A1,3,0.000028,732.375933,0.073238,0.000380
209,97C1,2,0.003111,78.547670,0.007855,0.396083
210,98A1,3,0.040872,751.036994,0.075104,0.544211
211,98D1,3,0.090510,779.240642,0.077924,1.161512


In [101]:
sdw_all_agg = (
    sdw_all_agg[["unique_id", "tonnes_dry_matter_ha"]]
    .groupby("unique_id")
    .sum()
    .reset_index()
)

In [102]:
sdw_all_agg.head(2)

,unique_id,tonnes_dry_matter_ha
0,110C1,0.001842
1,110D1,0.002858


In [103]:
sdw_all_agg.rename(
    columns={"tonnes_dry_matter_ha": "sdw_tonnes_dry_matter_ha"}, inplace=True
)

# Get total across all deadwood

In [104]:
sdw_all_agg.shape

(190, 2)

In [105]:
stumps_agg.shape

(440, 2)

In [106]:
ldw_agg.shape

(476, 2)

In [107]:
deadwood = vmd0002_eq9(stumps_agg, ldw_agg, sdw_all_agg)

In [108]:
deadwood.head(2)

,unique_id,stumps_tonnes_dry_matter_ha,ldw_tonnes_dry_matter_ha,sdw_tonnes_dry_matter_ha,all_tonnes_dry_matter_ha,tC_per_ha,CO2e_per_ha
0,100A1,NaN,0.207976,NaN,0.207976,0.097749,0.358412
1,100B1,NaN,1.091449,NaN,1.091449,0.512981,1.880930


In [109]:
deadwood.rename(
    columns={"CO2e_per_ha": "deadwood_CO2e_per_ha", "tC_per_ha": "deadwood_tC_per_ha"},
    inplace=True,
)

In [110]:
deadwood.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   unique_id                    569 non-null    object 
 1   stumps_tonnes_dry_matter_ha  440 non-null    float64
 2   ldw_tonnes_dry_matter_ha     476 non-null    float64
 3   sdw_tonnes_dry_matter_ha     190 non-null    float64
 4   all_tonnes_dry_matter_ha     569 non-null    float64
 5   deadwood_tC_per_ha           569 non-null    float64
 6   deadwood_CO2e_per_ha         569 non-null    float64
dtypes: float64(6), object(1)
memory usage: 31.2+ KB


## Export and upload data

In [111]:
# Upload to BQ
if len(deadwood) != 0:
    deadwood.to_csv(CARBON_STOCK_OUTDIR / "deadwood_carbon_stock.csv", index=False)
    pandas_gbq.to_gbq(
        deadwood,
        f"{DATASET_ID}.deadwood_carbon_stock",
        project_id=GCP_PROJ_ID,
        if_exists=IF_EXISTS,
        progress_bar=True,
    )
else:
    raise ValueError("Dataframe is empty.")

100%|██████████| 1/1 [00:00<00:00, 7397.36it/s]
